<a href="https://colab.research.google.com/github/priya123a/pest-control-system-in-greenhouse-/blob/main/project_int345.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/project_345/project/video.mp4" # Path to your video
FRAME_DIR = "/content/drive/MyDrive/Colab Notebooks/project_345/project/frames"  # Directory to save frames
IMG_SIZE = (224, 224)  # Resize frames to the model input size

# Create directory for frames if it doesn't exist
if not os.path.exists(FRAME_DIR):
    os.makedirs(FRAME_DIR)

# Function to extract frames from video
def extract_frames(video_path, frame_dir, img_size=(224, 224)):
    video = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Resize and save the frame
        frame_resized = cv2.resize(frame, img_size)
        frame_name = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(frame_dir, frame_name)
        cv2.imwrite(frame_path, frame_resized)
        frame_count += 1

    video.release()
    print(f"Frames extracted and saved to {frame_dir}")

# Extract frames from the video
extract_frames(VIDEO_PATH, FRAME_DIR, IMG_SIZE)




Frames extracted and saved to /content/drive/MyDrive/Colab Notebooks/project_345/project/frames


In [ ]:

def preprocess(input_folder, output_folder, size=(640, 640)):

    os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(input_folder, file_name)
            img = cv2.imread(img_path)

            # Resize
            img = cv2.resize(img, size)

            # Noise removal
            denoised = cv2.GaussianBlur(img, (5, 5), 0)

            # Histogram equalization
            ycrcb = cv2.cvtColor(denoised, cv2.COLOR_BGR2YCrCb)
            ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
            img_eq = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

            # Convert to RGB
            img_rgb = cv2.cvtColor(img_eq, cv2.COLOR_BGR2RGB)

            # Save result
            out_path = os.path.join(output_folder, file_name)
            cv2.imwrite(out_path, cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
input_path="/content/drive/MyDrive/Colab Notebooks/project_345/project/frames"
output_path="/content/drive/MyDrive/Colab Notebooks/project_345/project/preprocessing"
preprocess(input_path, output_path)

In [ ]:
import os
from ultralytics import YOLO

# Load the pre-trained YOLO model
model = YOLO("yolov5su.pt")  # replace with your custom model path if needed

# Input and output folder paths
frame_folder = "/content/drive/MyDrive/Colab Notebooks/project_345/project/preprocessing"
output_folder = "/content/drive/MyDrive/Colab Notebooks/project_345/project/output"
os.makedirs(output_folder, exist_ok=True)

# Process each frame
for file_name in os.listdir(frame_folder):
    if file_name.lower().endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(frame_folder, file_name)

        # Print progress
        print(f"Processing {file_name}...")

        # Run inference
        results = model(img_path)

        # Check if detections were made
        if len(results[0].boxes) > 0:  # Ensure there are boxes detected
            # Save annotated image with bounding boxes
            save_path = os.path.join(output_folder, file_name)

            # Optional: Check if the file already exists to avoid overwriting
            if os.path.exists(save_path):
                base, ext = os.path.splitext(file_name)
                save_path = os.path.join(output_folder, f"{base}_processed{ext}")

            # Save the annotated image
            results[0].save(filename=save_path)
            print(f"Saved annotated result to {save_path}")
        else:
            print(f"No pests detected in {file_name}.")


Processing frame_0000.jpg...

image 1/1 /content/drive/MyDrive/Colab Notebooks/project_345/project/preprocessing/frame_0000.jpg: 640x640 1 person, 829.6ms
Speed: 3.4ms preprocess, 829.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Saved annotated result to /content/drive/MyDrive/Colab Notebooks/project_345/project/output/frame_0000_processed.jpg
Processing frame_0001.jpg...

image 1/1 /content/drive/MyDrive/Colab Notebooks/project_345/project/preprocessing/frame_0001.jpg: 640x640 (no detections), 1690.1ms
Speed: 17.3ms preprocess, 1690.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
No pests detected in frame_0001.jpg.
Processing frame_0002.jpg...

image 1/1 /content/drive/MyDrive/Colab Notebooks/project_345/project/preprocessing/frame_0002.jpg: 640x640 (no detections), 845.2ms
Speed: 4.0ms preprocess, 845.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
No pests detected in frame_0002.jpg.
Processing frame_0003.jpg...

ima

In [ ]:
#won't run here on collab, try on personal compiler
import winsound

output_folder = "/content/drive/MyDrive/Colab Notebooks/project_345/project/output"

if any(os.path.isfile(os.path.join(output_folder, f)) for f in os.listdir(output_folder)):
    winsound.Beep(1000, 500)
